### Import libraries.

In [1]:
import sqlite3
import json
import pandas as pd

### Use SQL library of choice. 

Used sqlite since it's saving locally from local runs. Can and will probably switch to SQLAlchemy for database url instance.

In [2]:
# creating file path
dbfile = '/Users/isaacmenchaca/Desktop/hnlSkeletonMTurk/participants.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)



[('assignments',), ('amt_hit',), ('campaign',), ('apscheduler_jobs',)]


### Query assignments database.

Contains info per participant. Display using pandas.

In [3]:
# Verify that you can query task data.
df = pd.read_sql_query("SELECT * FROM assignments", con)
display(df)

,uniqueid,assignmentid,workerid,hitid,ipaddress,browser,platform,language,cond,counterbalance,codeversion,beginhit,beginexp,endhit,bonus,status,mode,datastring
0,debugRRDS6U:debugFW6SAR,debugFW6SAR,debugRRDS6U,debugTKJDL4,127.0.0.1,safari,macos,en-us,0,0,0.0.1,2022-01-28 21:37:29.015721,2022-01-28 21:37:49.819768,2022-01-28 21:37:54.096406,0.0,3,debug,"{""condition"": 0, ""counterbalance"": 0, ""assignm..."


## What do the different status numbers mean? TODO: LOOK UP
1 instructions
2 end of instructions
3 end of experiment

### Verify data saved by PsiTurk.

If the data is there, your data saving method using psiturk is functional.

In [4]:
for i, datastring in enumerate(df['datastring'].values):
        print('Participant', i + 1)
        for participantData in json.loads(datastring)['data']:
            print(participantData['trialdata'])
        print()

Participant 1
{'phase': 'INSTRUCTIONS', 'templates': ['instructions/instruct-1.html', 'instructions/instruct-2.html', 'instructions/instruct-practiceStart.html'], 'action': 'Begin'}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-1.html', 'indexOf': 0, 'action': 'NextPage', 'viewTime': 1554}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-2.html', 'indexOf': 1, 'action': 'NextPage', 'viewTime': 747}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-practiceStart.html', 'indexOf': 2, 'action': 'FinishInstructions', 'viewTime': 983}
{'rt': 4539, 'stimulus': '<p style="font-size: 30px;">This is the practice trial. Press any key to begin.</p>', 'response': ' ', 'trial_type': 'html-keyboard-response', 'trial_index': 0, 'time_elapsed': 4542, 'internal_node_id': '0.0-0.0'}
{'rt': 4381, 'answer': 'lessThan50', 'response': 'greaterThan50', 'correct': False, 'trial_type': 'DisplayTableStim', 'trial_index': 1, 'time_elapsed': 8926, 'internal_node_id': '0.0-1.0

### Test custom code for assigning workers bonuses.

Test how you will scrape the data and insert into custom.py code.

In [5]:
for i, datastring in enumerate(df['datastring'].values):
        print('Participant', i + 1)
        bonus = 0
        for participantData in json.loads(datastring)['data']:
            if "trial_type" in participantData['trialdata'].keys():
                if participantData['trialdata']['trial_type'] == 'DisplayTableStim':
                    
                    print(participantData['trialdata'])
                    if participantData['trialdata']['correct']:
                        bonus += 0.02
                        
        print('Participant bonus:', bonus)
        print()

Participant 1
{'rt': 1732, 'answer': 'lessThan50', 'response': 'lessThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 1, 'time_elapsed': 2442, 'internal_node_id': '0.0-1.0'}
{'rt': 1561, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 4, 'time_elapsed': 5866, 'internal_node_id': '0.0-4.0'}
{'rt': 1726, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 5, 'time_elapsed': 7598, 'internal_node_id': '0.0-5.0'}
{'rt': 1411, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 6, 'time_elapsed': 9016, 'internal_node_id': '0.0-6.0'}
{'rt': 1227, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 7, 'time_elapsed': 10249, 'internal_node_id': '0.0-7.0'}
Participant bonus: 0.1

Participant 

### Close connection to Database.

In [6]:
con.close()